<a href="https://colab.research.google.com/github/DefaultaideN/EncodeBoosting/blob/main/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Load some packages
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

print("Tensorflow Edition:", tf.__version__)
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)
print("Packages Loaded!")

Tensorflow Edition: 2.4.1
Packages Loaded!


In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2199.998
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4399.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [ ]:
!nvidia-smi

Fri Feb  5 08:00:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Load data from Google Drive
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#Define paths
DATA_PATH = '/content/drive/MyDrive/paper_file/data/'
SAVE_PATH = '/content/drive/MyDrive/paper_file/model_trained/'

In [ ]:
#Load pretrained models
model_loaded = []
for i in range(4):
  model = keras.models.load_model(SAVE_PATH + 'model_' + str(i))
  model_loaded.append(model)

In [ ]:
import shutil 
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
path = '/content'

data_loader = ImageDataGenerator

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import CSVLogger
from keras.layers import Dense
from keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
class Toget_transfer_model():
  def __init__(self, model_list, img_process_list, data_path, save_path, name_list):
    self.model_list = model_list
    self.img_process_list = img_process_list
    self.save_path = save_path
    self.data_path = data_path
    self.labels = []
    self.name_list = name_list

  def load_data(self, index, batch_size=16):
    data_path = self.data_path
    single_model = self.model_list[index]
    single_process = self.img_process_list[index]

    input_shape = single_model.layers[0].input_shape[0][1:3]
    image_loader = ImageDataGenerator(preprocessing_function = single_process)
    aug_loader = ImageDataGenerator(
      preprocessing_function = single_process,
      #rescale=1./255,
      rotation_range=40,
      #width_shift_range=0.2,
      #height_shift_range=0.2,
      #shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      vertical_flip=True,
      channel_shift_range=10
      )
    
    print('Train dataset:')
    trainaug_loader = aug_loader.flow_from_directory(directory= data_path+'train', target_size=input_shape, batch_size=batch_size)
    print('Validation dataset:')
    val_loader = image_loader.flow_from_directory(directory= data_path+'val', target_size=input_shape, batch_size=batch_size) 

    return trainaug_loader, val_loader


  def transfer_train_with_model(self, i, train_aug, val_loader, name='model_default', epoch=20):
    name = 'model_'+str(i)
    model = self.model_list[i]
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    if not os.path.exists(self.save_path + name):
      os.mkdir(self.save_path + name)

    checkpoint_filepath = self.save_path + name
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_filepath,
        save_weights_only = False,
        monitor = 'val_accuracy',
        mode = 'max',
        save_best_only = True)
    csv_logger = CSVLogger(self.save_path + name + '/train_history.csv', append=False)

    model.fit(x = train_aug,
          steps_per_epoch = len(train_aug),
          epochs = epoch,
          verbose = 2, 
          validation_data = val_loader,
          callbacks = [model_checkpoint_callback, csv_logger]
          ) 

    return model  
  

  def freeze_layers(self, index, layer_to_freeze):
    model = self.model_list[index]
    for layer in model.layers[:layer_to_freeze]:
      layer.trainable = False
    def count_params(model):
      non_trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.non_trainable_weights])
      trainable_params = np.sum([np.prod(v.get_shape().as_list()) for v in model.trainable_weights])
      return {'non_trainable_params': non_trainable_params, 'trainable_params': trainable_params}

    print(count_params(model))#Show number of trainable parameters
    return model
  

  def reshape_model(self, index, encode_shape, layer_to_remove):
    model = self.model_list[index]
    x = model.layers[layer_to_remove-1].output
    flatten = Flatten(name='fla_encode')(x)
    encode = Dense(units=encode_shape, activation='relu', name='encode')(flatten)
    output = Dense(units=4, activation='softmax', name='new_predictions')(encode)
    return keras.Model(inputs=model.input, outputs=output)


  def print_test(self, index):
    single_model = self.model_list[index]
    single_process = self.img_process_list[index]
    input_shape = single_model.layers[0].input_shape[0][1:3]
    image_loader = ImageDataGenerator(preprocessing_function = single_process)

    test_loader = image_loader.flow_from_directory(directory= self.data_path+'test', target_size=input_shape, batch_size=1578, shuffle=False)
    print('Testing model_{} ...'.format(index))
    results = single_model.evaluate(test_loader)
    print('model_{}: test_loss:{:.3f}, test_auc:{:.3f}\n'.format(index, results[0], results[1]))  

  def print_predict(self, index):
    single_model = self.model_list[index]
    single_process = self.img_process_list[index]
    input_shape = single_model.layers[0].input_shape[0][1:3]
    image_loader = ImageDataGenerator(preprocessing_function = single_process)

    test_loader = image_loader.flow_from_directory(directory= self.data_path+'test', target_size=input_shape, batch_size=1578, shuffle=False)
    print('Predict using model_{} ...'.format(index))
    X,y = test_loader.next()
    results = single_model.predict(X)
    #print(results)  
    return X, y, results

  def plot_decompose(self, encode, index, labels):
    model_name = 'model_'+str(index)+'_'+self.name_list[index]
    from sklearn.manifold import TSNE
    decompiser_tsne = TSNE(n_components=2)
    de_tsne = decompiser_tsne.fit_transform(encode)

    fig, ax = plt.subplots()
    fig.set_size_inches(7, 7)
    scatter = ax.scatter(x=de_tsne[:,0], y=de_tsne[:,1], c=labels)
    legend1 = ax.legend(*scatter.legend_elements(), loc="best", title="Classes")
    ax.set_title(model_name+'---tsne')
    ax.add_artist(legend1)

    from sklearn.manifold import Isomap
    decompiser_iso = Isomap(n_components=2, n_neighbors=10)
    de_iso = decompiser_iso.fit_transform(encode)

    fig, ax = plt.subplots()
    fig.set_size_inches(7, 7)
    scatter = ax.scatter(x=de_iso[:,0], y=de_iso[:,1], c=labels)
    legend1 = ax.legend(*scatter.legend_elements(), loc="best", title="Classes")
    ax.set_title(model_name+'---Isomap')
    ax.add_artist(legend1)


  def get_decompose(self, index):
    single_model = self.model_list[index]
    single_process = self.img_process_list[index]
    input_shape = single_model.layers[0].input_shape[0][1:3]

    image_loader = ImageDataGenerator(preprocessing_function = single_process)
    test_loader = image_loader.flow_from_directory(directory= self.data_path+'test', shuffle=False,
                            target_size=input_shape, batch_size=16)
    labels = np.array([])
    for step in range(len(test_loader)):
      X, y = test_loader.next()
      labels = np.append(labels, y.argmax(axis=1))
    
    encoder = keras.Model(inputs=[self.model_list[index].input],
              outputs=[self.model_list[index].layers[-2].output])
    single_model = encoder
    single_process = self.img_process_list[index]
    input_shape = single_model.layers[0].input_shape[0][1:3]
    image_loader = ImageDataGenerator(preprocessing_function = single_process)

    model_encode = single_model.predict(test_loader)
    self.plot_decompose(model_encode, index, labels)

    return model_encode


In [ ]:
from tensorflow import keras

model_VGG19 = keras.applications.vgg19
model_XCEP = keras.applications.xception
model_EFCb5 = keras.applications.efficientnet
model_RES50 = keras.applications.resnet
NAME_LIST = ['VGG', 'Xcept', 'Efcb5', 'Res50']
model_list = [model_VGG19, model_XCEP, model_EFCb5, model_RES50]

img_process = [model.preprocess_input for model in model_list]
#model_list = [model_VGG19.VGG19(), model_XCEP.Xception(), model_EFCb5.EfficientNetB5(), model_RES50.ResNet50()]
test_loaded_model = Toget_transfer_model(model_loaded, img_process, DATA_PATH, SAVE_PATH, NAME_LIST)


In [ ]:
for index in range(4):
  print('Testing model: '+NAME_LIST[index])
  test_loaded_model.print_test(index)
  print('----------------------------------------------------')


In [ ]:
X, labels, results = test_loaded_model.print_predict(0)

Found 1578 images belonging to 4 classes.
Predict using model_0 ...


In [ ]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(labels, results, average='macro', multi_class='ovr')

In [ ]:
labels[0]

array([1., 0., 0., 0.], dtype=float32)

In [ ]:
from numpy import argmax
label_index = []
result_index = []
for label in labels:
  label_index.append(argmax(label))
for result in results:
  result_index.append(argmax(result))

In [ ]:
from sklearn.metrics import confusion_matrix
cnf_matrix = confusion_matrix(label_index, result_index)
cnf_matrix

array([[294,   3,   3,  18],
       [ 10, 207,   1,  26],
       [  0,   0, 519,   3],
       [  9,   8,   6, 471]])

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix
from numpy import argmax
from numpy import mean
for model_i in range(4):
  X, labels, results = test_loaded_model.print_predict(model_i)
  label_index = []
  result_index = []
  for label in labels:
    label_index.append(argmax(label))
  for result in results:
    result_index.append(argmax(result))

  cnf_matrix = confusion_matrix(label_index, result_index)
  #print(cnf_matrix)
  #[[1 1 3]
  # [3 2 2]
  # [1 3 1]]

  FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
  FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
  TP = np.diag(cnf_matrix)
  TN = cnf_matrix.sum() - (FP + FN + TP)

  FP = FP.astype(float)
  FN = FN.astype(float)
  TP = TP.astype(float)
  TN = TN.astype(float)

  # Sensitivity, hit rate, recall, or true positive rate
  TPR = TP/(TP+FN)
  # Specificity or true negative rate
  TNR = TN/(TN+FP) 
  # Precision or positive predictive value
  PPV = TP/(TP+FP)
  # Negative predictive value
  NPV = TN/(TN+FN)
  # Fall out or false positive rate
  FPR = FP/(FP+TN)
  # False negative rate
  FNR = FN/(TP+FN)
  # False discovery rate
  FDR = FP/(TP+FP)
  # Overall accuracy
  ACC = (TP+TN)/(TP+FP+FN+TN)
  print('Model: '+NAME_LIST[model_i]+' | '+'Precision: '+str(PPV)+' | '+'Specificity: '+str(TNR))

Found 1578 images belonging to 4 classes.
Predict using model_0 ...
Model: VGG | Precision: [0.93929712 0.94954128 0.98109641 0.90926641] | Specificity: [0.98492063 0.99175412 0.9905303  0.95664207]
Found 1578 images belonging to 4 classes.
Predict using model_1 ...
Model: Xcept | Precision: [0.95583596 0.92436975 0.98484848 0.94141414] | Specificity: [0.98888889 0.98650675 0.99242424 0.97324723]
Found 1578 images belonging to 4 classes.
Predict using model_2 ...
Model: Efcb5 | Precision: [0.96226415 0.95378151 0.98674242 0.96153846] | Specificity: [0.99047619 0.99175412 0.99337121 0.98247232]
Found 1578 images belonging to 4 classes.
Predict using model_3 ...
Model: Res50 | Precision: [0.94193548 0.92241379 0.97748593 0.92842942] | Specificity: [0.98571429 0.98650675 0.98863636 0.96678967]


In [ ]:
mean(TNR), mean(PPV)

(0.9809617818275331, 0.9448003066470746)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import matthews_corrcoef

for i in range(4):  
  #print('iter: '+ str(i))
  X, labels, results = test_loaded_model.print_predict(i)
  score = roc_auc_score(labels, results, average='micro',multi_class='ovr')
  print('Model '+str(i)+' AUC: '+str(score)+ ' | MCC: '+str(mcc))

Found 1578 images belonging to 4 classes.
Predict using model_0 ...


KeyboardInterrupt: ignored

In [ ]:
from sklearn.metrics import roc_auc_score
from numpy import argmax
for i in range(4):
  X, labels, results = test_loaded_model.print_predict(i)
  onehot_results = results
  for row in range(results.shape[0]):
    index = argmax(results[row])
    for column in range(4):
      if column == index:
        onehot_results[row, column] = bool(True)
      else:
        onehot_results[row, column] = bool(False)
  score = roc_auc_score(labels, onehot_results, average='macro')
  print('Model: '+ NAME_LIST[index] + '| ' + 'AUC: ' + str(score) +'|')

Found 1578 images belonging to 4 classes.
Predict using model_0 ...
Model: Res50| AUC: 0.9747596153846154|
Found 1578 images belonging to 4 classes.
Predict using model_1 ...
Model: Efcb5| AUC: 0.9604166666666667|
Found 1578 images belonging to 4 classes.
Predict using model_2 ...
Model: Res50| AUC: 0.9299679487179486|
Found 1578 images belonging to 4 classes.
Predict using model_3 ...
Model: Xcept| AUC: 1.0|
